# Test of native TF/Keras implementation

In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, SpatialDropout1D, BatchNormalization, Dropout
from tensorflow.keras.metrics import AUC

2021-11-27 15:39:01.683979: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-27 15:39:01.684032: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Setup Synthetic Data

In [2]:
# setup for reproducible data
np.random.seed(13)

#create artificial training data
X = np.random.normal(size = (10000, 10, 30))
y = np.random.binomial(1, 0.3, size = 10000)

In [3]:
print(f'X shape {X.shape}, y shape {y.shape}')

X shape (10000, 10, 30), y shape (10000,)


##  test original 3-d Structure

In [4]:
# setup for reproducible training
tf.keras.backend.clear_session()
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

np.random.seed(1919)
tf.random.set_seed(1919)
random.seed(1919)

In [5]:
# In[ ]:


#build and compile TF model


def nn():
    model_input = Input(shape = (10, 30))
    model_dropout1 = SpatialDropout1D(0.2)(model_input)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [6]:
# In[ ]:

model = nn()
model.fit(X, y, epochs=5)



2021-11-27 15:39:03.531109: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-27 15:39:03.531160: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-27 15:39:03.531185: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dask-notebook-0): /proc/driver/nvidia/version does not exist
2021-11-27 15:39:03.531369: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
313/313 [==============================] - 9s 22ms/step - loss: 0.6910 - auc: 0.4970
Epoch 2/5
313/313 [==============================] - 6s 20ms/step - loss: 0.6445 - auc: 0.5044
Epoch 3/5
313/313 [==============================] - 6s 20ms/step - loss: 0.6287 - auc: 0.5177
Epoch 4/5
313/313 [==============================] - 6s 19ms/step - loss: 0.6218 - auc: 0.5212
Epoch 5/5
313/313 [==============================] - 6s 20ms/step - loss: 0.6156 - auc: 0.5386


## Test conversion to 2-d structure

In [7]:
# reset all TF state
tf.keras.backend.clear_session()
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

In [8]:
# setup for reproducible training
np.random.seed(1919)
tf.random.set_seed(1919)
random.seed(1919)

In [9]:
# setup model for 2-d structure


def nn2():
    model_input = Input(shape = (10 * 30))   # altered for 2-d structure
    model_input_unpack = tf.reshape(model_input, (-1, 10, 30))  # unpack to 3-d structure
    model_dropout1 = SpatialDropout1D(0.2)(model_input_unpack)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [10]:
model2 = nn2()

X_2d = X.reshape(-1, 10 * 30)

model2.fit(X_2d, y, epochs=5)

Epoch 1/5
313/313 [==============================] - 9s 21ms/step - loss: 0.6910 - auc: 0.4970
Epoch 2/5
313/313 [==============================] - 6s 19ms/step - loss: 0.6445 - auc: 0.5044
Epoch 3/5
313/313 [==============================] - 6s 19ms/step - loss: 0.6287 - auc: 0.5177
Epoch 4/5
313/313 [==============================] - 6s 20ms/step - loss: 0.6218 - auc: 0.5212
Epoch 5/5
313/313 [==============================] - 6s 18ms/step - loss: 0.6156 - auc: 0.5386


## Compared learned parameters from the two models

In [11]:
## Compare learned parameters
for nn_parameter, nn2_parameter in zip(model.weights, model2.weights):
    print(np.all(np.isclose(nn_parameter.numpy(), nn2_parameter.numpy())))

True
True
True
True
True
True
True
True
True
True
True


## Test 2-d structure with scikeras.KerasClassifer

In [12]:
from scikeras.wrappers import KerasClassifier

In [13]:
# reset all TF state
tf.keras.backend.clear_session()
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

In [14]:
# setup for reproducible training
np.random.seed(1919)
tf.random.set_seed(1919)
random.seed(1919)

In [15]:
# setup model for 2-d structure


def nn_scikeras():
    model_input = Input(shape = (10 * 30))   # altered for 2-d structure
    model_input_unpack = tf.reshape(model_input, (-1, 10, 30))  # unpack to 3-d structure
    model_dropout1 = SpatialDropout1D(0.2)(model_input_unpack)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [16]:
model_scikeras = KerasClassifier(model=nn_scikeras, optimizer='Adam')

X_2d = X.reshape(-1, 10 * 30)

model_scikeras.fit(X_2d, y, epochs=5)

Epoch 1/5
313/313 [==============================] - 9s 21ms/step - loss: 0.6910 - auc: 0.4970
Epoch 2/5
313/313 [==============================] - 7s 21ms/step - loss: 0.6445 - auc: 0.5044
Epoch 3/5
313/313 [==============================] - 7s 21ms/step - loss: 0.6287 - auc: 0.5177
Epoch 4/5
313/313 [==============================] - 6s 20ms/step - loss: 0.6218 - auc: 0.5212
Epoch 5/5
313/313 [==============================] - 6s 19ms/step - loss: 0.6156 - auc: 0.5386


KerasClassifier(
	model=<function nn_scikeras at 0x7fe1c6554af0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=Adam
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

## Compared learned parameters from the two models

In [17]:
## Compare learned parameters
for nn_parameter, scikeras_parameter in zip(model.weights, model_scikeras.model_.weights):
    print(np.all(np.isclose(nn_parameter.numpy(), scikeras_parameter.numpy())))

True
True
True
True
True
True
True
True
True
True
True
